In [20]:
import sys
import os

# This ensures Python can locate the cnnClassifier module from research/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))


In [21]:
%pwd

'C:\\Projects\\Chicken_Disease_Classification'

In [15]:
os.chdir("C:\Projects\Chicken_Disease_Classification")


In [16]:
%pwd

'C:\\Projects\\Chicken_Disease_Classification'

In [22]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
import sys
import importlib
from cnnClassifier.constants import *
import cnnClassifier.utils.common as common

# Force reload
importlib.reload(common)

# Check what’s available now

from cnnClassifier.utils.common import read_yaml, create_directories


['Any', 'BoxValueError', 'ConfigBox', 'Path', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'base64', 'create_directories', 'decodeImage', 'encodeImageIntoBase64', 'ensure_annotations', 'get_size', 'joblib', 'json', 'load_bin', 'load_json', 'logger', 'os', 'read_yaml', 'save_bin', 'save_json', 'yaml']


In [37]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [38]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [39]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [41]:
#pipeline creation
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-06-03 02:42:28,984: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-03 02:42:28,992: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-03 02:42:29,000: INFO: common: created directory at: artifacts]
[2025-06-03 02:42:29,003: INFO: common: created directory at: artifacts/data_ingestion]


[2025-06-03 02:42:33,759: INFO: 1170291011: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 39EA:333CB2:449EB6:A6A295:683E13BF
Accept-Ranges: bytes
Date: Mon, 02 Jun 2025 21:12:32 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100034-HYD
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1748898752.117301,VS0,VE837
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: a3bbb4646e1a9fc08171cb1c3b85d922d43d9fbe
Expires: Mon, 02 Jun 2025 21:17:32 GMT
Source-Age: 1

]
